In [ ]:
pip install transformers datasets seqeval --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
pip install --upgrade transformers

In [3]:
import json
from datasets import Dataset

with open("ner_dataset_50_cleaned.json", "r", encoding="utf-8") as f:
    data = json.load(f)

dataset = Dataset.from_list(data)
dataset = dataset.train_test_split(test_size=0.2)

print(dataset)
dataset["train"][0]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 24
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 6
    })
})


{'tokens': ['Space',
  'triangles',
  'for',
  'hangers',
  '12pcs',
  'ዋጋ፦',
  '400ብር',
  'ውስን',
  'ፍሬ',
  'ነው',
  'ያለው',
  'አድራሻ',
  'መገናኛመሰረትደፋርሞልሁለተኛፎቅ',
  'ቢሮ',
  'ቁ',
  'S05',
  'S06',
  '0902660722',
  '0928460606',
  'በTelegram',
  'ለማዘዝ',
  'ይጠቀሙ',
  '@zemencallcenter',
  '@zemenexpressadmin',
  'ለተጨማሪ',
  'ማብራሪያ',
  'የቴሌግራም',
  'ገፃችን',
  'https',
  'telegram',
  'me',
  'zemenexpress'],
 'ner_tags': ['B-PRODUCT',
  'I-PRODUCT',
  'I-PRODUCT',
  'I-PRODUCT',
  'O',
  'O',
  'B-PRICE',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOCATION',
  'I-LOCATION',
  'I-LOCATION',
  'I-LOCATION',
  'I-LOCATION',
  'B-CONTACT',
  'I-CONTACT',
  'O',
  'O',
  'O',
  'B-CONTACT',
  'I-CONTACT',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O']}

In [4]:
from transformers import AutoTokenizer

labels = ["O", "B-PRODUCT", "I-PRODUCT", "B-PRICE", "I-PRICE", "B-LOCATION", "I-LOCATION", "B-CONTACT", "I-CONTACT"]
label2id = {l: i for i, l in enumerate(labels)}
id2label = {i: l for l, i in label2id.items()}

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [5]:
def tokenize_and_align_labels(example):
    tokenized = tokenizer(
        example["tokens"],
        is_split_into_words=True,
        return_offsets_mapping=True,
        padding="max_length",
        truncation=True,
        max_length=128  # <-- control sequence length here
    )
    labels = []
    word_ids = tokenized.word_ids()
    previous_word_idx = None

    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)
        elif word_idx != previous_word_idx:
            labels.append(label2id[example["ner_tags"][word_idx]])
            previous_word_idx = word_idx
        else:
            # For subwords of the same word, repeat the label (optional: use I-*)
            labels.append(label2id[example["ner_tags"][word_idx]])

    tokenized["labels"] = labels
    tokenized.pop("offset_mapping")  # we don't need this for training
    return tokenized


In [6]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=False)

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "xlm-roberta-base",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from seqeval.metrics import classification_report
from datasets import load_metric
import numpy as np

metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[p] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


/tmp/ipython-input-8-984906493.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [9]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./ner_xlm_model",
    do_train=True,
    do_eval=True,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


/tmp/ipython-input-10-669581489.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: meronmuluye (meronmuluye-24) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=60, training_loss=0.8222747802734375, metrics={'train_runtime': 794.7016, 'train_samples_per_second': 0.302, 'train_steps_per_second': 0.076, 'total_flos': 15678797598720.0, 'train_loss': 0.8222747802734375, 'epoch': 10.0})

In [11]:
trainer.save_model("xlmr-ner-amharic-ecommerce")
tokenizer.save_pretrained("xlmr-ner-amharic-ecommerce")


('xlmr-ner-amharic-ecommerce/tokenizer_config.json',
 'xlmr-ner-amharic-ecommerce/special_tokens_map.json',
 'xlmr-ner-amharic-ecommerce/sentencepiece.bpe.model',
 'xlmr-ner-amharic-ecommerce/added_tokens.json',
 'xlmr-ner-amharic-ecommerce/tokenizer.json')

In [2]:
import json

with open("tokenized_messages.json", "r", encoding="utf-8") as f:
    unlabeled_data = json.load(f)


In [15]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 581.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 486.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 957.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [4]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

model_path = "xlmr-ner-amharic-ecommerce"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)
model.eval()

XLMRobertaForTokenClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768

In [13]:
from tqdm import tqdm

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load data
with open("tokenized_messages.json", "r", encoding="utf-8") as f:
    messages = json.load(f)

label_map = model.config.id2label

def batch_predict(batch_tokens):
    encodings = tokenizer(batch_tokens, is_split_into_words=True, return_tensors="pt", padding=True, truncation=True)
    word_ids_batch = [encodings.word_ids(i) for i in range(len(batch_tokens))]

    inputs = {k: v.to(device) for k, v in encodings.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=2)

    results = []
    for i, preds_i in enumerate(preds):
        word_ids = word_ids_batch[i]
        pred_labels = []
        prev_word_id = None
        for idx, word_id in enumerate(word_ids):
            if word_id is None or word_id == prev_word_id:
                continue
            label_id = preds_i[idx].item()
            pred_labels.append(label_map[label_id])
            prev_word_id = word_id
        results.append(pred_labels)
    return results

# Batch processing
auto_labeled = []
batch_size = 16

for i in tqdm(range(0, len(messages), batch_size)):
    batch = messages[i:i+batch_size]
    tokens_batch = [m["tokens"] for m in batch]
    pred_tags_batch = batch_predict(tokens_batch)
    for tokens, tags in zip(tokens_batch, pred_tags_batch):
        auto_labeled.append({
            "tokens": tokens,
            "ner_tags": tags
        })

# Save output
with open("auto_labeled.json", "w", encoding="utf-8") as f:
    json.dump(auto_labeled, f, ensure_ascii=False, indent=2)


100%|██████████| 342/342 [50:41<00:00,  8.89s/it]


In [14]:
with open("auto_labeled.json", "w", encoding="utf-8") as f:
    json.dump(auto_labeled, f, ensure_ascii=False, indent=2)

In [ ]:
import json

with open(ner_dataset_50_cleaned.json)", "r", encoding="utf-8") as f1:
    manual_data = json.load(f1)

with open("auto_labeled.json", "r", encoding="utf-8") as f2:
    auto_data = json.load(f2)

# Optional: remove duplicates
combined_data = manual_data + auto_data
unique_data = []
seen = set()

for item in combined_data:
    key = tuple(item["tokens"])
    if key not in seen:
        seen.add(key)
        unique_data.append(item)

# Save merged data
with open("merged_labeled_data.json", "w", encoding="utf-8") as out:
    json.dump(unique_data, out, ensure_ascii=False, indent=2)

print(f"Merged dataset size: {len(unique_data)}")


In [ ]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Load merged data
with open("merged_labeled_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Train/test split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

dataset = DatasetDict({
    "train": Dataset.from_list(train_data),
    "test": Dataset.from_list(test_data)
})

In [ ]:
label_list = ["O", "PRODUCT", "PRICE", "LOCATION", "DELIVERY_FEE", "CONTACT_INFO"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}


In [ ]:
def tokenize_and_align_labels(example):
    tokenized = tokenizer(
        example["tokens"],
        is_split_into_words=True,
        return_offsets_mapping=True,
        padding="max_length",
        truncation=True,
        max_length=128
    )
    labels = []
    word_ids = tokenized.word_ids()
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)
        elif word_idx != previous_word_idx:
            labels.append(label2id[example["ner_tags"][word_idx]])
            previous_word_idx = word_idx
        else:
            labels.append(label2id[example["ner_tags"][word_idx]])
    tokenized["labels"] = labels
    return tokenized


In [ ]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=False)

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "xlm-roberta-base",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir="./xlmr-ner-v2",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=6,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir="./logs"
)

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)
    true_labels, pred_labels = [], []

    for pred, lab in zip(predictions, labels):
        for p_i, l_i in zip(pred, lab):
            if l_i != -100:
                true_labels.append(id2label[l_i])
                pred_labels.append(id2label[p_i])

    from sklearn.metrics import classification_report, f1_score
    print(classification_report(true_labels, pred_labels))
    return {"f1": f1_score(true_labels, pred_labels, average="weighted")}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()